In [ ]:
import pandas as pd
import pandas_read_xml as pdx
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
import xml.etree.ElementTree as ET
import pandas as pd

xml_data = open('./train.xml', 'r', encoding = 'utf-8').read() 
root = ET.XML(xml_data)  

data = []
cols = []

for i, child in enumerate(root):
    data.append([subchild.text for subchild in child])
    cols.append(child.tag)

df_train = pd.DataFrame(data)  
df_train.columns = ['movie', 'review_id', 'name','user_id','note','commentaire']  

In [ ]:

xml_data = open('./dev.xml', 'r', encoding = 'utf-8').read() 
root = ET.XML(xml_data)  
data = []
cols = []

for i, child in enumerate(root):
    data.append([subchild.text for subchild in child])
    cols.append(child.tag)

df_dev = pd.DataFrame(data)  
df_dev.columns = ['movie', 'review_id', 'name','user_id','note','commentaire'] 

In [ ]:

xml_data = open('./test.xml', 'r', encoding = 'utf-8').read() 
root = ET.XML(xml_data)  
data = []
cols = []

for i, child in enumerate(root):
    data.append([subchild.text for subchild in child])
    cols.append(child.tag)

df_test = pd.DataFrame(data)  
df_test.columns = ['movie', 'review_id', 'name','user_id','commentaire'] 

In [ ]:
import emoji
import nltk
from string import digits

import seaborn as sns
import time

import re
from spellchecker import SpellChecker
spell = SpellChecker(language='fr')
tokenizer = nltk.RegexpTokenizer(r'''\w'|\w+|[^\w\s]''')
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer(language='french')
import spacy
nlp = spacy.load("fr_core_news_sm")


In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

In [ ]:
# Enlèvement des lignes manquantes

In [ ]:
sd = df_dev.isin(['',None,np.nan])
lignes_Manquants = sd[sd.eq(True).any(1)]
lignes_Manquants.index
df_dev.drop(df_dev.index[lignes_Manquants.index],inplace = True)
df_dev.index = range(df_dev.shape[0])

In [ ]:
sd = df_train.isin(['',None,np.nan])
lignes_Manquants = sd[sd.eq(True).any(1)]
lignes_Manquants.index
df_train.drop(df_train.index[lignes_Manquants.index],inplace = True)
df_train.index = range(df_train.shape[0])

In [ ]:
for i in range(len(df_train)):
    df_train['note'][i] = df_train['note'][i].replace(',', '.')# remplacer la virgule par point 
    
for i in range(len(df_dev)):
    df_dev['note'][i] = df_dev['note'][i].replace(',', '.')

In [ ]:
# remplacer les emojis avec de text
for i in range(df_train.shape[0]):
    if text_has_emoji(df_train['commentaire'].iloc[i]):
        df_train['commentaire'][i] = emoji.demojize(df_train['commentaire'][i],language = 'fr')
for i in range(df_dev.shape[0]):
    if text_has_emoji(df_dev['commentaire'].iloc[i]):
        df_dev['commentaire'][i] = emoji.demojize(df_dev['commentaire'][i],language = 'fr') 
        
for i in range(df_test.shape[0]):
    if text_has_emoji(df_test['commentaire'].iloc[i]):
        df_test['commentaire'][i] = emoji.demojize(df_test['commentaire'][i],language = 'fr') 

In [ ]:
#la tokénisation
tokenized_sentences_train = []
tokenized_sentences_dev = []
tokenized_sentences_test = []
tokenizer = nltk.RegexpTokenizer(r'''\w'|\w+|[^\w\s]''')
for i in range(len(df_train)):
    tokenized_sentences_train.append(tokenizer.tokenize(df_train['commentaire'][i]))
    
for i in range(len(df_dev)):
    tokenized_sentences_dev.append(tokenizer.tokenize(df_dev['commentaire'][i]))
    
for i in range(len(df_train)):
    tokenized_sentences_test.append(tokenizer.tokenize(df_test['commentaire'][i]))

In [ ]:
#construire une liste qui contient un ou deux caractères
words_twochar = []
for i in tokenized_sentences_train:
    for j in i:
        if len(j)<3:
            words_twochar.append(j)
            

for i in tokenized_sentences_dev:
    for j in i:
        if len(j)<3:
            words_twochar.append(j)


for i in tokenized_sentences_test:
    for j in i:
        if len(j)<3:
            words_twochar.append(j)

In [ ]:
#les stops words
sw = set()
ignore_words = ['?', '!',',','.',';','-','_']
stop_words = nltk.corpus.stopwords.words('french')

sw.update(ignore_words)
sw.update(stop_words)
sw.update(words_twochar)

In [ ]:
for i in range(len(df_train)):
    l_temp = []
    try:
        tokens = tokenizer.tokenize(df_train['commentaire'][i])
        for item in tokens:
            if item not in sw:
                l_temp.append(item)
        df_train['commentaire'][i] = " ".join(l_temp)
    except:
        continue
        
for i in range(len(df_test)):
    l_temp = []
    try:
        tokens = tokenizer.tokenize(df_test['commentaire'][i])
        for item in tokens:
            if item not in sw:
                l_temp.append(item)
        df_test['commentaire'][i] = " ".join(l_temp)
    except:
        continue
        
for i in range(len(df_dev)):
    l_temp = []
    try:
        tokens = tokenizer.tokenize(df_dev['commentaire'][i])
        for item in tokens:
            if item not in sw:
                l_temp.append(item)
        df_dev['commentaire'][i] = " ".join(l_temp)
    except:
        continue

In [ ]:
def return_stem(sentence):
    doc = nlp(sentence)
    return [stemmer.stem(X.text) for X in doc]

In [ ]:

def get_clean(x):
    x = str(x).lower().replace('\\', '').replace('_', ' ')
    x = ps.cont_exp(x) 
    x = ps.remove_emails(x)
    x = ps.remove_urls(x)
    x = ps.remove_accented_chars(x)
    x = ps.remove_special_chars(x)
    x = re.sub("(.)\\1{2,}", "\\1", x)
    x = " ".join(return_stem(x))
    return x

In [ ]:
!pip install git+https://github.com/laxmimerit/preprocess_kgptalkie.git --upgrade --force-reinstall # 

In [ ]:
import preprocess_kgptalkie as ps #importation des pretraitement prêtes mais nous avons utilisé seulement celle qui marche sur la langue française aussi

In [ ]:
for i in range(len(df_train)):
    df_train['commentaire'][i] = get_clean(df_train['commentaire'][i])
    
for i in range(len(df_dev)):
    df_dev['commentaire'][i] = get_clean(df_dev['commentaire'][i])
    
for i in range(len(df_test)):
    df_dev['commentaire'][i] = get_clean(df_test['commentaire'][i])

In [ ]:
#enlèevement des valeurs none après les prétraitement effectués

In [ ]:
non_values_train = []
for i in range(len(df_train['commentaire'])):
    if df_train['commentaire'][i] is np.nan:
        non_values_train.append(i)

non_values_dev = []
for i in range(len(df_dev['commentaire'])):
    if df_dev['commentaire'][i] is np.nan:
        non_values_dev.append(i)

In [ ]:
for i in non_values_dev:
    df_dev = df_dev.drop(index=i, axis = 0)
df_dev = df_dev.reset_index(drop=True)

for i in non_values_train:
    df_train = df_train.drop(index=i, axis = 0)
df_train = df_train.reset_index(drop=True)

In [ ]:
for i in range(len(df_train)): #convertir les à des entiers
    df_train['note'][i] = (df_train['note'][i])*2
    if i < len(df_dev):
        df_dev['note'][i] = (df_dev['note'][i])*2

In [ ]:
#Enlèvement des chiffres

In [ ]:
from string import digits

In [ ]:
list_string_dev = df_dev['commentaire'].values.tolist()
output_string_dev = []
remove_digits = str.maketrans('', '', digits)
try:
    for item in list_string_dev:
        output_string_dev.append(item.translate(remove_digits))
except:
    print(item)

In [ ]:
list_string_train = df_train['commentaire'].values.tolist()
output_string_train = []
remove_digits = str.maketrans('', '', digits)
try:
    for item in list_string_train:
        output_string_train.append(item.translate(remove_digits))
except:
    print(item)

In [ ]:
list_string_test = df_test['commentaire'].values.tolist()
output_string_test = []
remove_digits = str.maketrans('', '', digits)
try:
    for item in list_string_test:
        output_string_test.append(item.translate(remove_digits))
except:
    print(item)

In [ ]:
df_dev['commentaire'] = output_string_dev
df_train['commentaire'] = output_string_train
df_test['commentaire'] = output_string_test

In [ ]:
df_train['commentaire'].isnull().sum()

In [ ]:
df_dev['commentaire'].isnull().sum()

In [ ]:
df_dev.to_csv("C_dev.csv",index = False)

In [ ]:
df_train.to_csv("C_train.csv",index = False)

In [ ]:
df_test.to_csv("C_train.csv",index = False)